# Import

In [1]:
%install-swiftpm-flags -c release
%install '.package(url: "https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git", from: "0.0.6")' SwiftCoreMLTools
%install '.package(url: "https://github.com/dduan/Just.git", from: "0.8.0")' Just

Installing packages:
	.package(url: "https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git", from: "0.0.6")
		SwiftCoreMLTools
	.package(url: "https://github.com/dduan/Just.git", from: "0.8.0")
		Just
With SwiftPM flags: ['-c', 'release']
Working in: /tmp/tmpam7mbnoa/swift-install
Fetching https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git
Fetching https://github.com/dduan/Just.git
Fetching https://github.com/apple/swift-protobuf.git
Cloning https://github.com/apple/swift-protobuf.git
Resolving https://github.com/apple/swift-protobuf.git at 1.8.0
Cloning https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git
Resolving https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git at 0.0.6
Cloning https://github.com/dduan/Just.git
Resolving https://github.com/dduan/Just.git at 0.8.0
[1/3] Compiling Just Just.swift
[2/3] Compiling SwiftProtobuf AnyMessageStorage.swift
[3/4] Compiling SwiftCoreMLTools Activations.swift
[4/5] Compiling jupyterInstalledPackages jupyterInstalled

In [2]:
import Foundation
import SwiftCoreMLTools
import TensorFlow
import Just

# Data Download

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per ten thousand dollars
        - PTRATIO  pupil-teacher ratio by town
        - B        1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
        - LSTAT    % lower status of the population
        - MEDV     Median value of owner-occupied homes in a thousand dollar

    :Missing Attribute Values: None

    :Creator: Harrison, D. and Rubinfeld, D.L.

This is a copy of UCI ML housing dataset.
https://archive.ics.uci.edu/ml/machine-learning-databases/housing/


In [3]:
// if let cts = Just.get(URL(string: "https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data")!).content {
//     try! cts.write(to: URL(fileURLWithPath:"../data/housing.csv"))
// }

# Data Ingestion

In [4]:
let data = try String(contentsOfFile:"./data/housing.csv", encoding: String.Encoding.utf8)
let dataRecords: [[Float]] = data.split(separator: "\n").map{ String($0).split(separator: " ").compactMap{ Float(String($0)) } }

let numRecords = dataRecords.count
let numColumns = dataRecords[0].count

let dataFeatures = dataRecords.map{ Array($0[0..<numColumns-1]) }
let dataLabels = dataRecords.map{ Array($0[(numColumns-1)...]) }

# Data Transformation

## Split Numerical Categorical Features

In [5]:
let categoricalColumns = [3, 8]
let numericalColumns = [0, 1, 2, 4, 5, 6, 7, 9, 10, 11, 12]
let numCategoricalFeatures = categoricalColumns.count
let numNumericalFeatures = numericalColumns.count
let numLabels = 1

assert(numColumns == numCategoricalFeatures + numNumericalFeatures + 1)

// Get Categorical Features
let allCategoriesValues = dataFeatures.map{ row in categoricalColumns.map{ Int32(row[$0]) } }
                                .reduce(into: Array(repeating: [Int32](), count: 2)){ total, value in
                                    total[0].append(value[0])
                                    total[1].append(value[1]) }
                                .map{ Set($0).sorted() }

let embeddingSizes = allCategoriesValues.map{ $0.count }

let categoricalFeatures = dataFeatures.map{ row in categoricalColumns.map{ Int32(row[$0]) } }
let oneHotCategoricalFeatures:[[[Int32]]] = categoricalFeatures.map{ catArray in
    var oneHotArray = [[Int32]]()
    
    for i in 0..<catArray.count {
        var oneHot = Array(repeating: Int32(0), count: allCategoriesValues[i].count)
        if let pos = allCategoriesValues[i].firstIndex(where: { $0 == catArray[i] }){
            oneHot[pos] = 1
        }
        oneHotArray.append(oneHot)
    }
    
    return oneHotArray
}

// Get Numerical Features
let numericalFeatures = dataFeatures.map{ row in numericalColumns.map{ row[$0] } }

## Split Train and Test

In [6]:
let trainPercentage:Float = 0.8
let numTrainRecords = Int(ceil(Float(numRecords) * trainPercentage))
let numTestRecords = numRecords - numTrainRecords

func matrixTranspose<T>(_ matrix: [[T]]) -> [[T]] {
    if matrix.isEmpty {return matrix}
    var result = [[T]]()
    for index in 0..<matrix.first!.count {
        result.append(matrix.map{$0[index]})
    }
    return result
}

let xCategoricalAllTrain = matrixTranspose(Array(oneHotCategoricalFeatures[0..<numTrainRecords])).map{ Array($0.joined()) }
let xCategoricalAllTest = matrixTranspose(Array(oneHotCategoricalFeatures[numTrainRecords...])).map{ Array($0.joined()) }
let xNumericalAllTrain = Array(Array(numericalFeatures[0..<numTrainRecords]).joined())
let xNumericalAllTest = Array(Array(numericalFeatures[numTrainRecords...]).joined())
let yAllTrain = Array(Array(dataLabels[0..<numTrainRecords]).joined())
let yAllTest = Array(Array(dataLabels[numTrainRecords...]).joined())

let XCategoricalTrain = xCategoricalAllTrain.enumerated().map{ (offset, element) in 
    Tensor<Int32>(element).reshaped(to: TensorShape([numTrainRecords, embeddingSizes[offset]]))
}
let XCategoricalTest = xCategoricalAllTest.enumerated().map{ (offset, element) in 
    Tensor<Int32>(element).reshaped(to: TensorShape([numTestRecords, embeddingSizes[offset]]))
}
let XNumericalTrainDeNorm = Tensor<Float>(xNumericalAllTrain).reshaped(to: TensorShape([numTrainRecords, numNumericalFeatures]))
let XNumericalTestDeNorm = Tensor<Float>(xNumericalAllTest).reshaped(to: TensorShape([numTestRecords, numNumericalFeatures]))
let YTrain = Tensor<Float>(yAllTrain).reshaped(to: TensorShape([numTrainRecords, numLabels]))
let YTest = Tensor<Float>(yAllTest).reshaped(to: TensorShape([numTestRecords, numLabels]))

## Normalize Numerical Features

In [7]:
let mean = XNumericalTrainDeNorm.mean(alongAxes: 0)
let std = XNumericalTrainDeNorm.standardDeviation(alongAxes: 0)

print(mean, std)

[[ 2.0137098,  14.197531,   9.523555, 0.53213036,  6.3311296,   64.47929,  4.1678762,  353.68396,
    18.03163,  379.84735,  11.394517]] [[ 6.5076075,  25.258776,   6.534038, 0.11449408,  0.7311985,  29.000755,  2.1797554,  132.14561,
    2.217345,  40.494495,   6.852825]]


In [8]:
let XNumericalTrain = (XNumericalTrainDeNorm - mean)/std
let XNumericalTest = (XNumericalTestDeNorm - mean)/std

In [9]:
print("Training shapes \(XNumericalTrain.shape) \(XCategoricalTrain[0].shape) \(XCategoricalTrain[1].shape) \(YTrain.shape)")
print("Testing shapes  \(XNumericalTest.shape) \(XCategoricalTest[0].shape) \(XCategoricalTest[1].shape) \(YTest.shape)")

Training shapes [405, 11] [405, 2] [405, 9] [405, 1]
Testing shapes  [101, 11] [101, 2] [101, 9] [101, 1]


# Model

In [26]:
struct MultiInputs<N: Differentiable, C>: Differentiable {
  var numerical: N
  
  @noDerivative
  var categorical: C

  @differentiable
  init(numerical: N, categorical: C) {
    self.numerical = numerical
    self.categorical = categorical
  }
}

struct RegressionModel: Module {
//     var numericalLayer = Dense<Float>(inputSize: 11, outputSize: 32, activation: relu)
    var embedding1 = TensorFlow.Embedding<Float>(vocabularySize: 2, embeddingSize: 2)
    var embedding2 = TensorFlow.Embedding<Float>(vocabularySize: 9, embeddingSize: 5)
//     var embeddingLayer = Dense<Float>(inputSize: (4 + 45), outputSize: 64, activation: relu)
//     var allInputConcatLayer = Dense<Float>(inputSize: (32 + 64), outputSize: 128, activation: relu)
    var allInputConcatLayer = Dense<Float>(inputSize: (11 + 4 + 45), outputSize: 128, activation: relu)
    var hiddenLayer = Dense<Float>(inputSize: 128, outputSize: 32, activation: relu)
    var outputLayer = Dense<Float>(inputSize: 32, outputSize: 1)
    
    @differentiable
    func callAsFunction(_ input: MultiInputs<[Tensor<Float>], [Tensor<Int32>]>) -> Tensor<Float> {
//         let numericalInput = numericalLayer(input.numerical[0])
        let embeddingOutput1 = embedding1(input.categorical[0])
        
        print(input.categorical[0].shape, embeddingOutput1.shape)
        
        let embeddingOutput1Reshaped = embeddingOutput1.reshaped(to: 
            TensorShape([embeddingOutput1.shape[0], embeddingOutput1.shape[1] * embeddingOutput1.shape[2]]))
        let embeddingOutput2 = embedding2(input.categorical[1])
        let embeddingOutput2Reshaped = embeddingOutput2.reshaped(to: 
            TensorShape([embeddingOutput2.shape[0], embeddingOutput2.shape[1] * embeddingOutput2.shape[2]]))
//         let embeddingConcat = Tensor<Float>(concatenating: [embeddingOutput1Reshaped, embeddingOutput2Reshaped], alongAxis: 1)
//         let embeddingInput = embeddingLayer(embeddingConcat)
//         let allConcat = Tensor<Float>(concatenating: [numericalInput, embeddingInput], alongAxis: 1)
        let allConcat = Tensor<Float>(concatenating: [input.numerical[0], embeddingOutput1Reshaped, embeddingOutput2Reshaped], alongAxis: 1)
        return allConcat.sequenced(through: allInputConcatLayer, hiddenLayer, outputLayer)
    }
}

var model = RegressionModel()

# Training

In [27]:
let optimizer = RMSProp(for: model, learningRate: 0.001)
Context.local.learningPhase = .training

In [28]:
let epochCount = 500
let batchSize = 32
let numberOfBatch = Int(ceil(Double(numTrainRecords) / Double(batchSize)))
let shuffle = true

func mae(predictions: Tensor<Float>, truths: Tensor<Float>) -> Float {
    return abs(Tensor<Float>(predictions - truths)).mean().scalarized()
}

In [29]:
for epoch in 1...epochCount {
    var epochLoss: Float = 0
    var epochMAE: Float = 0
    var batchCount: Int = 0
    var batchArray = Array(repeating: false, count: numberOfBatch)
    for batch in 0..<numberOfBatch {
        var r = batch
        if shuffle {
            while true {
                r = Int.random(in: 0..<numberOfBatch)
                if !batchArray[r] {
                    batchArray[r] = true
                    break
                }
            }
        }
        
        let batchStart = r * batchSize
        let batchEnd = min(numTrainRecords, batchStart + batchSize)
        let (loss, grad) = model.valueWithGradient { (model: RegressionModel) -> Tensor<Float> in
            let multiInput = MultiInputs(numerical: [XNumericalTrain[batchStart..<batchEnd]],
                                         categorical: [XCategoricalTrain[0][batchStart..<batchEnd],
                                                       XCategoricalTrain[1][batchStart..<batchEnd]])
            let logits = model(multiInput)
            return meanSquaredError(predicted: logits, expected: YTrain[batchStart..<batchEnd])
        }
        optimizer.update(&model, along: grad)
        
        let multiInput = MultiInputs(numerical: [XNumericalTrain[batchStart..<batchEnd]],
                                     categorical: [XCategoricalTrain[0][batchStart..<batchEnd],
                                                   XCategoricalTrain[1][batchStart..<batchEnd]])
        let logits = model(multiInput)
        epochMAE += mae(predictions: logits, truths: YTrain[batchStart..<batchEnd])
        epochLoss += loss.scalarized()
        batchCount += 1
    }
    epochMAE /= Float(batchCount)
    epochLoss /= Float(batchCount)

    print("Epoch \(epoch): MSE: \(epochLoss), MAE: \(epochMAE)")
}

[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[21, 2] [21, 2, 2]
[21, 2] [21, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
Epoch 1: MSE: 513.012, MAE: 20.06003
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[21, 2] [21, 2, 2]
[21, 2] [21, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 

[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
Epoch 18: MSE: 22.487621, MAE: 2.8686776
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[21, 2] [21, 2, 2]
[21, 2] [21, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
Epoch 19: MSE: 21.254286, MAE: 2.8620846
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2

[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
Epoch 34: MSE: 17.550167, MAE: 2.4954114
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[21, 2] [21, 2, 2]
[21, 2] [21, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
Epoch 35: MSE: 17.001442, MAE: 2.447809
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[21, 2] [21, 2, 2]
[21, 2]

[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[21, 2] [21, 2, 2]
[21, 2] [21, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
Epoch 51: MSE: 13.88611, MAE: 2.1314108
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[21, 2] [21, 2, 2]
[21, 2] [21, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
Epoch 52: MSE: 13.598247, MAE: 2.1030416
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2]

[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
Epoch 67: MSE: 11.318426, MAE: 1.8872056
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[21, 2] [21, 2, 2]
[21, 2] [21, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
Epoch 68: MSE: 10.652658, MAE: 1.880878
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[21, 2] [21, 2, 2]
[21, 2] [21, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2]

...

[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
Epoch 420: MSE: 1.9986769, MAE: 0.97878325
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[21, 2] [21, 2, 2]
[21, 2] [21, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
Epoch 421: MSE: 2.850806, MAE: 1.229046
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[21, 2] [21, 2, 2]
[21, 

[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
Epoch 436: MSE: 2.7326398, MAE: 1.2144104
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[21, 2] [21, 2, 2]
[21, 2] [21, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
Epoch 437: MSE: 2.6000686, MAE: 1.1196988
[21, 2] [21, 2, 2]
[21, 2] [21, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32,

[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
Epoch 452: MSE: 2.2866693, MAE: 1.2103292
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[21, 2] [21, 2, 2]
[21, 2] [21, 2, 2]
Epoch 453: MSE: 2.1374292, MAE: 1.2295916
[21, 2] [21, 2, 2]
[21, 2] [21, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32,

[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
Epoch 468: MSE: 2.1282287, MAE: 1.2374759
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[21, 2] [21, 2, 2]
[21, 2] [21, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
Epoch 469: MSE: 2.451042, MAE: 1.270498
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[21, 2] [21, 2, 2]
[21, 2] [21, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2

[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
Epoch 484: MSE: 2.1976717, MAE: 1.3177027
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[21, 2] [21, 2, 2]
[21, 2] [21, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
Epoch 485: MSE: 2.0549333, MAE: 1.0904158
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[21, 2] [21, 2, 2]
[21, 2] [21, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32, 2] [32, 2, 2]
[32,

# Test

In [15]:
Context.local.learningPhase = .inference

let multiInputTest = MultiInputs(numerical: [XNumericalTest],
                                 categorical: [XCategoricalTest[0],
                                               XCategoricalTest[1]])

let prediction = model(multiInputTest)

let predictionMse = meanSquaredError(predicted: prediction, expected: YTest).scalarized()/Float(numTestRecords)
let predictionMae = mae(predictions: prediction, truths: YTest)/Float(numTestRecords)

print("MSE: \(predictionMse), MAE: \(predictionMae)")

MSE: 0.44058493, MAE: 0.045618672


# Export

In [16]:
print(model.embedding1.embeddings.shape, model.embedding2.embeddings.shape)

[2, 2] [9, 5]


In [24]:
let coremlModel = Model(version: 4,
                        shortDescription: "Regression",
                        author: "Jacopo Mangiavacchi",
                        license: "MIT",
                        userDefined: ["SwiftCoremltoolsVersion" : "0.0.6"]) {
    Input(name: "numericalInput", shape: [11])
    Input(name: "categoricalInput1", shape: [2])
    Input(name: "categoricalInput2", shape: [9])
    Output(name: "output", shape: [1])
    NeuralNetwork {
        Embedding(name: "embedding1",
                     input: ["categoricalInput1"],
                     output: ["outEmbedding1"],
                     weight: model.embedding1.embeddings.transposed().flattened().scalars,
                     inputDim: 2,
                     outputChannels: 2)
        Permute(name: "permute1",
                     input: ["outEmbedding1"],
                     output: ["outPermute1"],
                     axis: [2, 1, 0, 3])
        Flatten(name: "flatten1",
                     input: ["outPermute1"],
                     output: ["outFlatten1"],
                     mode: .last)
        Embedding(name: "embedding2",
                     input: ["categoricalInput2"],
                     output: ["outEmbedding2"],
                     weight: model.embedding2.embeddings.transposed().flattened().scalars,
                     inputDim: 9,
                     outputChannels: 5)
        Permute(name: "permute2",
                     input: ["outEmbedding2"],
                     output: ["outPermute2"],
                     axis: [2, 1, 0, 3])
        Flatten(name: "flatten2",
                     input: ["outPermute2"],
                     output: ["outFlatten2"],
                     mode: .last)
        Concat(name: "concat",
                     input: ["numericInput", "outFlatten1", "outFlatten2"],
                     output: ["outConcat"])
        InnerProduct(name: "dense1",
                     input: ["outConcat"],
                     output: ["outDense1"],
                     weight: model.allInputConcatLayer.weight.transposed().flattened().scalars,
                     bias: model.allInputConcatLayer.bias.flattened().scalars,
                     inputChannels: 60,
                     outputChannels: 128)
        ReLu(name: "Relu1",
             input: ["outDense1"],
             output: ["outRelu1"])
        InnerProduct(name: "dense2",
                     input: ["outRelu1"],
                     output: ["outDense2"],
                     weight: model.hiddenLayer.weight.transposed().flattened().scalars,
                     bias: model.hiddenLayer.bias.flattened().scalars,
                     inputChannels: 128,
                     outputChannels: 32)
        ReLu(name: "Relu2",
             input: ["outDense2"],
             output: ["outRelu2"])
        InnerProduct(name: "dense3",
                     input: ["outRelu2"],
                     output: ["output"],
                     weight: model.outputLayer.weight.transposed().flattened().scalars,
                     bias: model.outputLayer.bias.flattened().scalars,
                     inputChannels: 32,
                     outputChannels: 1)
    }
}

In [25]:
let coreMLData = coremlModel.coreMLData
try! coreMLData!.write(to: URL(fileURLWithPath: "./s4tf_house_simplified_trained_model.mlmodel"))